# APLICACIONES EN CIENCIAS DE COMPUTACION (INF265)¶
Dr. Edwin Villanueva Talavera (ervillanueva@pucp.edu.pe)

# Practica: Agente solucionador de problemas - entorno "4-puzzle"  

El presente notebook muestra una implementacion de un Agente de Resolución de Problemas para el entorno de 4-puzzle (3 piezas numeradas del 1 al 3 y una pocision vacia). Un Algoritmo general de búsqueda con memoria de nodos expandidos ya se encuentra implementado (graph_search).
Al final del notebook deberas responder a las preguntas planteadas. 

![VacuumEnvironment](4-puzzle.png)


# Implementación del entorno 4-puzzle

#### Clase <b>Thing</b>
Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>. (No editar)  

In [ ]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

#### Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [ ]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepcion que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una accion 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localizacion especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningun agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localizacion location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

#### Clase <b>FourpuzzleEnvironment</b>

Esta clase implementa el entorno específico del 4-puzzle. Un agente en este entorno percibe el estado del entorno, el cual es representado por un string de 4 caracteres: los dos primeros caracteres representan las posiciones de arriba y los dos ultimos caracteres las posiciones de abajo. El caracter * representa la posición en blanco. Falta implementar execute_action()

In [ ]:
class FourpuzzleEnvironment(Environment):

    def __init__(self, initial_state):
        super().__init__()
        self.status = initial_state
        
    def thing_classes(self):
        return [FourpuzzleReflexAgent]

    def percept(self, agent):
        """Retorna el estado del ambiente (las piezas que estan en cada posicion)"""
        return self.status

    def execute_action(self, agent, action):
        """Implementa el MAPA De TRANSICION: Cambia la posicion de las piezas de acuerdo a la accion solicitada del blanco; 
        Cada accion valida provoca una disminución de desempeño en 1 unidad """
        
        state = list(self.status)
        iblank = state.index('*')    # obtiene el indice del casillero en blanco (representado con *)
        
        if action == 'Right':
            if (iblank%2 == 0):    # si la posicion del blanco esta en la izquierda (indice par) -> intercambia con la derecha
                state[iblank], state[iblank+1] = state[iblank+1], state[iblank]
                agent.performance -= 1
        elif action == 'Left':
            if (iblank%2 != 0):    # si la posicion del blanco esta en la derecha (indice impar) -> intercambia con la izquierda
                state[iblank-1], state[iblank] = state[iblank], state[iblank-1]
                agent.performance -= 1
        elif action == 'Up':
            if (iblank > 1):    # si la posicion del blanco esta abajo (indice > 1) -> intercambia con el de arriba
                state[iblank-2], state[iblank] = state[iblank], state[iblank-2]
                agent.performance -= 1
        elif action == 'Down':
            if (iblank <= 1):    # si la posicion del blanco esta arriba (indice <=1) -> intercambia con el de abajo
                state[iblank], state[iblank+2] = state[iblank+2], state[iblank]
                agent.performance -= 1    
                
        self.status = ''.join(state) 
  

# Implementacion del Problema de Busqueda y Algoritmos de Busqueda


#### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.


In [ ]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state (Modelo de transicion).
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

    def value(self, state):
        """En problemas de optimizacion, cada estado tiene un valor. Algoritmos
        como Hill-climbing intentan maximizar este valor."""
        raise NotImplementedError

#### <b> Clase FourpuzzleSearchProblem </b>  
Esta es una subclase de SearchProblem donde se define el problema de busqueda especifico para el ambiente del 4-puzzle. 

In [ ]:
class FourpuzzleSearchProblem(SearchProblem):

    def actions(self, state):
        """Retorna las acciones ejecutables desde el estado state.
        Por ejemplo, para el estado '32*1' debe retornar: acciones = ['Up', 'Right']"""
        acciones = []

        iblank = state.index('*')    # obtiene el indice del casillero en blanco (representado con *) del estado
        if iblank == 0:
            acciones = ['Right','Down']
        if iblank == 1:
            acciones = ['Left','Down']
        if iblank == 2:
            acciones = ['Right','Up']
        if iblank == 3:
            acciones = ['Left','Up']
        
        return acciones

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion action desde state.
        La accion debe ser alguna de self.actions(state)
        Por ejemplo, para  state='32*1' y action='Up' debe retornar newState = '*231' """  
        
        iblank = state.index('*')    # obtiene el indice del casillero en blanco (representado con *) del estado
        newState = list(state)          # copia state en una lista newState
        
        if action == 'Right':
            if (iblank%2 == 0):    # si la posicion del blanco esta en la izquierda (indice par) -> intercambia con la derecha
                newState[iblank], newState[iblank+1] = newState[iblank+1], newState[iblank]
        elif action == 'Left':
            if (iblank%2 != 0):    # si la posicion del blanco esta en la derecha (indice impar) -> intercambia con la izquierda
                newState[iblank-1], newState[iblank] = newState[iblank], newState[iblank-1]
        elif action == 'Up':
            if (iblank > 1):    # si la posicion del blanco esta abajo (indice > 1) -> intercambia con el de arriba
                newState[iblank-2], newState[iblank] = newState[iblank], newState[iblank-2]
        elif action == 'Down':
            if (iblank <= 1):    # si la posicion del blanco esta arriba (indice <=1) -> intercambia con el de abajo
                newState[iblank], newState[iblank+2] = newState[iblank+2], newState[iblank]

        return ''.join(newState)
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion es = 1"""
        return c + 1;

#### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

In [ ]:
class Node:

    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        return Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next_state))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

#### Clase <b>FIFOQueue</b> 
Define la clase que implementa una cola tipo FIFO First-In-First-Out (para BFS)

In [ ]:
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

#### <b>Algoritmo general de búsqueda con memoria de estados visitados (Graph Search)</b>

Busca en los sucesores de un problema para encontrar un objetivo. El argumento frontier debe ser una cola vacia. Si dos caminos arrivan al mismo estado, solo usa el primero.

In [ ]:
def graph_search(problem, frontier):

    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

# Implementacion del Agente de Resolucion del Problema 4-puzzle


#### Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [ ]:
import collections
import random

class Agent(Thing):
    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

#### <b>Clase FourpuzzleSearch </b> 
Esta clase implementa el programa del agente que planea y ejecuta una solucion en el entorno 4-puzzle

In [ ]:
class FourpuzzleSearchProgram:
    def __init__(self, goal_state, search_method):
        self.goal = goal_state
        self.method = search_method
        self.seq = []  # lista de acciones a ejecutar, inicialmente vacia
        
    def __call__(self, percept):
        state = percept
        if state == self.goal:    # Si el ambiente esta en el estado objetivo no hace nada
            return 'None'
        if not self.seq:  # si la lista de acciones esta vacia
            search_problem = FourpuzzleSearchProblem(state, self.goal)
            if self.method == 'bfs':
                goal_node = graph_search(search_problem, FIFOQueue()) # frontera es una cola FIFO 
            elif self.method == 'dfs':
                goal_node = graph_search(search_problem, []) # frontera es una pila ([] es una pila en Python)
            else:
                raise NotImplementedError
                        
            if goal_node == None: # sin solucion
                print('No se encontro solucion para el 4-puzzle con metodo {}'.format(self.method) )
                return 'None'
            
            self.seq = goal_node.solution()
            print( 'Agente encontro una solucion al Problema de Busqueda 4-puzzle con {}: {}'.format(self.method, self.seq) )
        
        action = self.seq.pop(0)       
        return action 


# Probando el agente en 4-puzzle

In [ ]:
"""Crea un entorno del 4-puzzle con estado inicial '321*' """
e = FourpuzzleEnvironment('32*1')

# Imprime el estado inicial del ambiente
print("Estado Inicial del Ambiente: {}".format(e.status))

In [ ]:
"""Crea un agente de busqueda con estado objetivo '*123' y lo añade al entorno """
a = Agent( FourpuzzleSearchProgram ('*123', 'bfs') ) 

"""Añade el agente creado al entorno 4-puzzle"""
e.add_thing(a) 

In [ ]:
# Ejecuta 10 pasos del ambiente y obtiene el desempeño del agente
e.run(10)

# Imprime el estado actual del ambiente, y desempeño del agente
print("Estado del Ambiente despues de 10 pasos: {}".format(e.status))
print("FourpuzzleSearchAgent con desempeño = {}".format(a.performance))

# Preguntas:


1) <b>Ejecute BFS y DFS para los siguientes problemas:</b>

   - estado inicial = '\*231', estado objetivo = '\*123'
   - estado inicial = '3\*12', estado objetivo = '\*123'
   
   ¿Hay diferencias en los costos de las soluciones entre BFS y DFS? Explique Por qué y cual seria el método mas adecuado para este problema 

2) <b> ¿Todos los estados serán alcanzables en 4-puzzle?</b> verifique con el siguiente problema
   - estado inicial = '231\*', estado objetivo = '\*123'
   
   Intente dar una expliación de lo que esta sucediendo (talvez alguna particularidad del espacio de estados)

